# 自定义层

## 不带参数的层
**构造一个没有任何参数的自定义层**。  
下面的`CenteredLayer`类要从其输入中减去均值。要构建它，只需继承基础层类并实现前向传播功能。

In [1]:
import torch
import torch.nn.functional as F
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

In [2]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

可以[**将层作为组件合并到更复杂的模型中**]。

In [10]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())
# 健全性检查均值是否为0
Y = net(torch.rand(4, 8))
Y.mean()

tensor(0., grad_fn=<MeanBackward0>)

## 带参数的层
实现自定义版本的全连接层。该层需要两个参数，一个用于表示权重，另一个用于表示偏置项。  
在此实现中，使用修正线性单元作为激活函数。该层需要输入参数：`in_units`和`units`，分别表示输入数和输出数。  
用 **nn.Parameter()** 将初始化的值包起来就行

In [5]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units))
        
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [6]:
# 实例化MyLinear类并访问其模型参数
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[ 0.8315,  0.6255,  1.0919],
        [-1.0026,  1.0664, -0.8279],
        [ 1.2525,  1.4521, -1.5382],
        [ 1.1842, -0.5210, -0.0964],
        [ 0.2571, -0.3886,  0.4948]], requires_grad=True)

In [7]:
# 使用自定义层直接执行前向传播计算
linear(torch.rand(2, 5))

tensor([[0.6290, 1.2748, 0.0000],
        [0.0486, 0.3354, 0.0000]])

In [8]:
# 使用自定义层构建模型，就像使用内置的全连接层一样使用自定义层
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.0000],
        [1.5915]])